In [1]:
import json
import random
from collections import defaultdict
from openai import OpenAI
from tqdm import tqdm
import ast
import re
import time
import os
import pandas as pd
import glob
import os
import pprint
from itertools import product, islice
import tiktoken
import json
import hashlib
import nest_asyncio
import asyncio
import aiohttp
import time
from tqdm.asyncio import tqdm as tqdm_async
import json
from qdrant_client import QdrantClient
from qdrant_client import QdrantClient, models
import json
import gzip
from tqdm import tqdm
import os
import json
import gzip
import uuid
import time
import tiktoken
from tqdm import tqdm
from qdrant_client import QdrantClient, models
from openai import OpenAI, RateLimitError
from concurrent.futures import ThreadPoolExecutor
from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
import cohere
from concurrent.futures import ThreadPoolExecutor
import snowflake.connector
import re
import traceback
import os
import os
import traceback
import re
import snowflake.connector
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from helpers import open_ai_call, clean_and_parse_response, get_embedding, prepare_embeddings_and_keywords, get_qdrant_filters, boost_points, rerank_points_with_cohere, accumulate_qdrant_points, extract_top_points_info, build_prompt_from_points, clean_sql_query, extract_one_table_per_group, fetch_column_names, load_thread_cache, save_thread_cache, get_or_create_threads_for_db, create_prompt, create_new_threads, try_extract_sql, get_assistant_response, execute_sql_with_timeout, create_assistant, inject_sample_values_into_top_points, fetch_sample_values_from_column_lookup, normalize_column_names_with_lookup, get_column_names, fetch_sample_for_column, fetch_sample_values_from_column_lookup_parallel, fetch_columns_for_table, fetch_column_names_parallel

In [2]:
def inject_sample_values_into_top_points(top_points, sample_lookup):
    for item in top_points:
        db = item.get('database')
        schema = item.get('table')
        if not db or not schema:
            continue

        # Resolve table name
        if item.get('ungrouped_key') and item['ungrouped_key'] != 'N/A':
            table = item['ungrouped_key'].removesuffix('.json')
        else:
            template = item.get('template', '').removesuffix('.json')
            combinations = item.get('combinations', [])
            if not template or not combinations:
                continue
            first_combo = combinations[0]
            var_names = re.findall(r"\{(.*?)\}", template)
            if len(first_combo) != len(var_names):
                continue
            for var, val in zip(var_names, first_combo):
                template = template.replace(f"{{{var}}}", val)
            table = template

        lookup_key = (db.upper(), schema.upper(), table.upper())
        col = item.get('column_name')
        if not col:
            continue

        columns_dict = sample_lookup.get(lookup_key, {})

        '''print(f"\n🔍 Checking table {lookup_key} for column '{col}'")
        print(f"🧩 Available columns: {list(columns_dict.keys())}")
        print(f"🔍 Normalized: {[k.lower().strip() for k in columns_dict.keys()]}")'''

        matched_sample = columns_dict.get(col)

        if matched_sample:
            item['sample_values'] = matched_sample
        else:
            item['sample_values'] = 'N/A'
            print(f"⚠️ No match for {lookup_key} → column '{col}'")

    return top_points

def fetch_sample_for_column(conn_factory, db, schema, table, col, max_samples):
    try:
        conn = conn_factory()
        cs = conn.cursor()
        query = f'''
            SELECT DISTINCT "{col}"
            FROM {db}.{schema}.{table}
            WHERE "{col}" IS NOT NULL
            LIMIT {max_samples}
        '''
        cs.execute(query)
        values = [row[0] for row in cs.fetchall() if row[0] is not None]
        cs.close()
        conn.close()
        return ((db, schema, table, col), values)
    except Exception as e:
        print(f"⚠️ Failed to fetch values for {db}.{schema}.{table}.{col} → {e}")
        return ((db, schema, table, col), [])

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def fetch_sample_values_from_column_lookup_parallel(column_lookup, conn_factory, max_samples=10, max_workers=10):
    futures = []
    sample_lookup = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for (db, schema, table), columns in column_lookup.items():
            for col in columns:
                futures.append(
                    executor.submit(fetch_sample_for_column, conn_factory, db, schema, table, col, max_samples)
                )

        # Wrap as_completed with tqdm for progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching samples"):
            (db, schema, table, col), values = future.result()
            key = (db, schema, table)
            if values:
                if key not in sample_lookup:
                    sample_lookup[key] = {}
                sample_lookup[key][col] = values

    return sample_lookup

In [3]:
with open("/Users/paolocadei/Documents/Masters/Thesis/Spider2/api_keys/api_keys.json", "r") as f:
    api_keys = dict(json.load(f))

client = OpenAI(
        api_key = api_keys["open_ai_key"],
    )

qdrant_key =  api_keys["qdrant_key"]

qdrant_client = QdrantClient(
    url="https://17582e9e-4a04-4068-bf13-bd4fdc0d688d.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key=qdrant_key
)

cohere_key = api_keys["cohere"]
    
model = "gpt-4o-mini-2024-07-18"
EMBEDDING_MODEL = "text-embedding-3-small"

def conn_factory():
    return snowflake.connector.connect(
        user='pablo_ca',
        password=api_keys["snowflake"],
        account='RSRSBDK-YDB67606'
    )

In [5]:
while True:
    assistants = client.beta.assistants.list().data
    if not assistants:
        print("✅ All assistants deleted.")
        break

    for assistant in assistants:
        print(f"Deleting assistant: {assistant.name} ({assistant.id})")
        client.beta.assistants.delete(assistant.id)

checker_instructions = """
You are simulating a user interacting with an LLM to generate a Snowflake SQL query that answers a natural language question.

Your task is to evaluate the generated query and provide structured feedback. Follow this process:

---

1. Assess Relevance:
- Does the query logically and semantically answer the user's original question?
- Does it use appropriate columns, tables, and filters?

2. Check Syntax:
- Is the query valid Snowflake SQL?
- If there are syntax errors, identify them precisely.
- If the problem is with an INVALID IDENTIFIER, IT MOST LIKELY MEANS THAT THE COLUMN NAME WAS NOT REFERENCED CORRECTLY SO CHECK EACH COLUMN AND MAKE SURE THAT THEY ARE EXACTLY (EVEN THE CASE) AS THEY ARE IN THE INFORMATION PROVIDED.
  A LOT OF TIMES THE INVALID QUALIFIER IS DUE TO THE COLUMN NAME NOT BEING IN BETWEEN DOUBLE QUOTES (e.g. "column_name" instead of column_name).
- THE QUERIES NEED TO BE AS ROBUST AS POSSIBLE

---

3. Evaluate the Output (if present):
- Does the result make sense given the question?
- If the output is **null, empty, or full of zeros**, you must **never** assume the query is correct.
  - Investigate what caused the empty or meaningless result. This could be due to:
    - Missing or incorrect joins.
    - Overly restrictive or wrong WHERE clauses.
    - Invalid column references or filter logic.
    - Improper aggregations or GROUP BY conditions.
  - Remember: **there should always be a result**, unless the question explicitly calls for empty or filtered cases.

---

4. Provide Feedback:
Write a clear, concise paragraph to send back to the LLM. It should:

- Explain any issues in logic or syntax.
- Suggest how to fix them (clearly and specifically).
- If there are invalid identifiers, use the provided valid identifiers to correct the query.
- If a fix is obvious, write the revised query and return it as described below.
- If the query matches the user's intent **but returns no or zero results**, you must explain what may be causing this and request a fix.

---

5. Format for Output (main logic queries):

If you are **confident** you can fix the original query and generate a correct one, output only the query in this format:

SQL_QUERY_START
SELECT ...
FROM database.schema.table
WHERE ...
SQL_QUERY_END

⚠️ Important:
- Only generate a new query if the previous step did **not** already include one.
- Do **not** generate speculative or placeholder queries.
- Do **not** assume the model has access to runtime data—only evaluate queries and outputs as given.

---

6. Validate Output Columns:
- If the expected output column names are provided, ensure the query returns **only** those columns, in the correct order and format.
- Do not allow extra columns or missing columns.
- If needed, use column aliases (e.g., `AS "city_one"`) to match expected output exactly.
- If the output columns are incorrect, request that the query be rewritten with the correct `SELECT` clause.

---

7. Final Judgment:

If the query is **correct** and the **output is not empty**, and both align with the user’s question:
- Respond only with: `Query validated. Marking as final.`


If the query looks syntactically fine but the **output is empty**, that is **not sufficient** to mark it as final. You must identify what in the query might have led to this, or provide a standalone query to assist.

"""




sql_generator_instructions = """
You are a Snowflake SQL query generator. Your job is to generate a valid and efficient query using the **Snowflake SQL dialect** only.

Instructions:
- You will receive a natural language question and detailed database schema information.
- Your task is to generate a SQL query that accurately answers the question using only the provided schema.

Strict requirements:
1. Use only **Snowflake SQL dialect** — no functions or syntax from other dialects (e.g., no `LIMIT` without `FETCH`, no `TEXT[]`, no `ARRAY`, etc.).
2. The FROM clause must have three parts: `database.schema.table`. This information is given in the information provided as TABLE:
4. Do not assume columns or data unless explicitly described in the schema.
5. If using date filters like “one year ago”, use Snowflake syntax:
   `WHERE refresh_date BETWEEN DATEADD(year, -1, CURRENT_DATE()) AND CURRENT_DATE()`
6. Avoid hardcoded values when the question calls for reasoning (e.g., use calculations like midpoint/average for ranges).
7. Never use non-SQL logic or LLM-specific language (e.g., no `ARRAY_EXISTS`, `filter()`, or pseudo-code).
8. Use aliases and readable formatting when appropriate.
9. If the question asks for the id and you also have a column with the name, provide both of them in the SELECT statement (HAVING MORE COLUMNS IS BETTER THAN HAVING LESS COLUMNS).
10. MAKE SURE THAT THE COLUMNS NAMES ARE EXACTLY AS THEY ARE IN THE INFORMATION PROVIDED, IF THEY ARE IN LOWERCASE, WRAP THEM IN DOUBLE QUOTES (i.e. "column_name" instead of column_name).
11. DO NOT USE TO_TIMESTAMP_NTZ("column_name" / 1000000) BUT USE TO_TIMESTAMP("column_name" / 1000000)

MAKE SURE TO FOCUS ON THE CHANGES YOU ARE TOLD TO MAKE !

THE QUERIES NEED TO BE AS ROBUST AS POSSIBLE

Format:
- Wrap the final SQL query between these tags:

SQL_QUERY_START
SELECT ...
FROM database.schema.table
WHERE ...
SQL_QUERY_END

Goal:
Generate only one correct, executable Snowflake SQL query that directly answers the question using the provided schema, and that perfectly matches
Only return the SQL query between the tags. Do not include explanations, comments, or the question itself in the response.
"""


# Your system prompt
system_prompt = '''
You are a natural language assistant that reformulates analytical questions for use in a retrieval-augmented SQL generation system.

Your task is to take a user’s natural language question and generate 1 different reformulations that:
- Use varied phrasing and sentence structure
- Preserve the precise analytical intent of the original question
- Clearly express the entities, time logic, and conditions involved
- Are suitable for matching with table and column descriptions in a vector database

Each reformulation should be accurate, well-structured, and reflect the same logical meaning as the original using synonyms and restructuring the question completely are allowed (e.g. completed orders could be completed or delivered orders...)

NO TWO REFORMULATIONS SHOULD BE TOO SIMILAR. Each should be distinct in wording and structure while maintaining the original intent.
THE REFORMULATIONS SHOULD HAVE DIFFERENT WORDING AS MUCH AS POSSIBLE.

Return the output in the following JSON format:

{
"intent_paraphrases": [
    "Reformulation 1",
    ...
    "Reformulation 3"
]
}
'''

system_prompt2 = '''
You are an AI assistant designed to extract relevant entities, synonyms, metrics, time expressions, conditions, and other concepts from a user’s natural language question. These elements will be used for hybrid retrieval in a Qdrant vector database to match relevant database metadata (such as tables and columns) and assist with SQL query generation.

Please analyze the user's questions and extract the following:

1. **Entities**: General schema-relevant concepts implied in the question, such as "location", "date", "timestamp", "category", "score", etc should also be included (e.g. "last year" or "in the past 5 days" should result in "date" or "timestamp"). If any time-expressions are present, give them the label "date" or "timestamp".
2. **Synonyms**: Alternate or related terms that a user might use for the identified entities or other key concepts, which could help match to schema elements.
3. **Metrics**: Any quantitative concepts or aggregation methods mentioned, expressed as full phrases (e.g., "highest search score", "total count").
4. **Time Expressions**: Literal time-based phrases from the question (e.g., "one year ago", "last month", "from 2020 to 2023").
5. **Conditions**: Any filters, comparisons, or logical operations implied by the question (e.g., "greater than 100", "for California only").
6. **Other Concepts**: Additional meaningful terms or domain-specific ideas that help clarify the user’s intent (e.g., "top rising terms", "trending topics").

INPUT: A list of natural language questions, including the original and its reformulations.

Return your output in this JSON format:
{
"entities": [...],
"synonyms": [...],
"metrics": [...],
"time_expressions": [...],
"conditions": [...],
"other_keywords": [...]
}
'''

Deleting assistant: Snowflake Generator (asst_1wdQ2E6A8OUe8vEObHBsGXBN)
Deleting assistant: Snowflake Checker (asst_AZpQVxEls62W2x7yMhElZmVK)
Deleting assistant: Snowflake Generator (asst_OWgJWgbPHRAr3lpdDRYy6Cpd)
Deleting assistant: Snowflake Checker (asst_SIJvzKNTAmMrXPqVlvoeDquo)
Deleting assistant: Snowflake Generator (asst_nD5bKk2tPyKylFoMLPsQiNmR)
Deleting assistant: Snowflake Checker (asst_pUt0CUXwOkWQ50IMKk5NXggO)
Deleting assistant: Snowflake Generator (asst_7xN8S9frAyIrwwYqWoJS43X8)
Deleting assistant: Snowflake Checker (asst_9zdwYpOhBAMPdB4yMEe0i5vs)
Deleting assistant: Snowflake Generator (asst_FR9cnRY7klLlRMBCnnnGRu12)
Deleting assistant: Snowflake Checker (asst_wTeGqAC0ZKPjejBnbQobECxC)
Deleting assistant: Snowflake Generator (asst_YaMqJAdBaRoNpMhnDgGu8Dmx)
Deleting assistant: Snowflake Checker (asst_l7WM6jApsXMKP1DPx0lrmy7r)
Deleting assistant: Snowflake Generator (asst_OeqNlOVJIr8zYh12aTXTQSvc)
Deleting assistant: Snowflake Checker (asst_QPWCsSl0UXUNIDk5KZMfRDT1)
Deleti

In [35]:
lines = [
    #{"instance_id": "sf_bq395", "instruction": "Which 5 states' percentage change in unsheltered homeless individuals from 2015 to 2018 were top 5 closest to the national average? Please provide the state abbreviation.", "db_id": "SDOH", "external_knowledge": None},
    #{"instance_id": "sf_bq352", "instruction": "Please list the average number of prenatal weeks in 2018 for counties in Wisconsin where more than 5% of the employed population had commutes of 45-59 minutes in 2017.", "db_id": "SDOH", "external_knowledge": None},
    #{"instance_id": "sf_bq074", "instruction": "Count the number of counties that experienced an increase in unemployment from 2015 to 2018, using 5-year ACS data, and a decrease in dual-eligible enrollee counts between December 1, 2015, and December 1, 2018.", "db_id": "SDOH", "external_knowledge": None},
    #{"instance_id": "sf_bq066", "instruction": "Could you assess the relationship between the poverty rates from the previous year's census data and the percentage of births without maternal morbidity for the years 2016 to 2018? Use only data for births where no maternal morbidity was reported and for each year, use the 5-year census data from the year before to compute the Pearson correlation coefficient", "db_id": "SDOH", "external_knowledge": None},
    {"instance_id": "sf_local272", "instruction": "Which product ID, aisle, and position should be selected to pick the highest quantity for order 423, ensuring the picked quantity does not exceed the available inventory in warehouse 1, and calculate the quantity to be picked while prioritizing locations with earlier dates and smaller quantities?", "db_id": "ORACLE_SQL", "external_knowledge": None},
    {"instance_id": "sf_local273", "instruction": "What is the average pick percentage for each product (by name), considering the quantity picked from inventory locations that are ordered by the earliest purchase date and smallest quantity, while ensuring that the picked quantity matches the overlapping range between the order quantity and the available inventory?", "db_id": "ORACLE_SQL", "external_knowledge": None},
    {"instance_id": "sf_local274", "instruction": "Which products were picked for order 421, and what is the average number of units picked for each product, using FIFO (First-In, First-Out) method?", "db_id": "ORACLE_SQL", "external_knowledge": None},
    {"instance_id": "sf_local275", "instruction": "Which products (by name) had a seasonality-adjusted sales ratio consistently above 2 for the entire year of 2017, based on monthly sales data from January 2016?", "db_id": "ORACLE_SQL", "external_knowledge": "calculation_method.md"},
    {"instance_id": "sf_local277", "instruction": "What is the average forecasted annual sales for products 4160 and 7790 for 2018? Use a weighted regression model based on sales data from January 2016, focusing on the first 36 months, with sales adjusted for seasonality during time steps 7 to 30.", "db_id": "ORACLE_SQL", "external_knowledge": "calculation_method.md"},
    {"instance_id": "sf_local279", "instruction": "For each product, provide the product_id, month in 2019, and the smallest difference between its ending inventory and minimum required level, based on a monthly inventory adjustment model that includes restocking when levels fall below the minimum.", "db_id": "ORACLE_SQL", "external_knowledge": None},
    {"instance_id": "sf_local284", "instruction": "For veg whsle data, can you generate a summary of our items' loss rates? Include the average loss rate, and also break down the count of items that are below, above, and within one standard deviation from this average.", "db_id": "BANK_SALES_TRADING", "external_knowledge": None},
    {"instance_id": "sf_local286", "instruction": "Prepare a comprehensive performance report on our sellers, focusing on total sales, average item price, average review scores, and packing times. Ensure that the report includes only those sellers who have sold a quantity of more than 100 products and highlight the product category names in English with the highest sales volume.", "db_id": "ELECTRONIC_SALES", "external_knowledge": None},
    {"instance_id": "sf_local301", "instruction": "For weekly-sales data, I need an analysis of our sales performance around mid-June for the years 2018, 2019, and 2020. Specifically, calculate the percentage change in sales between the four weeks leading up to June 15 and the four weeks following June 15 for each year.", "db_id": "BANK_SALES_TRADING", "external_knowledge": None},
    {"instance_id": "sf_local302", "instruction": "Analyze the average sales performance impact 12 weeks before and after June 15, 2020, across various attributes like regions, platforms, age bands, demographics, and customer types. Identify and provide the attribute with the highest negative impact on sales, detailing the average percentage change in sales for that attribute.", "db_id": "BANK_SALES_TRADING", "external_knowledge": None},
    {"instance_id": "sf_local330", "instruction": "For each web page, how many unique user sessions either start or end there", "db_id": "LOG", "external_knowledge": None},
    {"instance_id": "sf_local331", "instruction": "List the three most common third actions users take after visiting the `/detail` page twice in a row, including each action's occurrence count.", "db_id": "LOG", "external_knowledge": None},
    {"instance_id": "sf_local358", "instruction": "How many users are there in each age category (20s, 30s, 40s, 50s, and others)?", "db_id": "LOG", "external_knowledge": None},
    {"instance_id": "sf_local344", "instruction": "How many times has each type of overtake occurred in Formula 1?", "db_id": "F1", "external_knowledge": "f1_overtake.md"},
    {"instance_id": "sf_local335", "instruction": "Which five constructors have had the most seasons in the 21st century where their drivers scored the fewest points in a Formula 1 season?", "db_id": "F1", "external_knowledge": None},
    {"instance_id": "sf_local310", "instruction": "List the three years where the sum of the highest points achieved by any driver and any constructor was the lowest", "db_id": "F1", "external_knowledge": None},
    {"instance_id": "sf_local356", "instruction": "Can you tell me the full names of drivers who have been overtaken more times than they have performed overtakes?", "db_id": "F1", "external_knowledge": None},
    {"instance_id": "sf003", "instruction": "From 2015 to 2020, which zip code in Census Zip Code Tabulation Areas had the second-highest annual population growth rate, given a minimum estimate of 25,000 people over a 5-year period? Include the zip code, state abbreviation, and growth rate.", "db_id": "GLOBAL_GOVERNMENT", "external_knowledge": None},
    {"instance_id": "sf006", "instruction": "Calculate the percentage change in the number of active financial branch entities for each state, comparing the counts on March 1, 2020, and December 31, 2021. Active entities are those operational on the specified dates.", "db_id": "FINANCE__ECONOMICS", "external_knowledge": None},
    {"instance_id": "sf008", "instruction": "Determine the percentage change in gross income inflow and the seasonally-adjusted purchase-only home price index for the Phoenix-Mesa-Scottsdale, AZ Metro Area from January 1, 2023, to December 31, 2023. Gross income inflow refers to the total adjusted gross income from all financial entities within the specified metro area", "db_id": "US_REAL_ESTATE", "external_knowledge": None},
    {"instance_id": "sf010", "instruction": "What are the cumulative percentages of mortgages near default in California for each recorded date in 2023, including those 90 to 180 days past due, in forbearance, or in the process of foreclosure, bankruptcy, or deed in lieu?", "db_id": "US_REAL_ESTATE", "external_knowledge": None},
    {"instance_id": "sf037", "instruction": "Calculate the shortest driving distance in miles between each 'The Home Depot' store, identified by its POI ID, and its nearest 'Lowe's Home Improvement' store", "db_id": "US_REAL_ESTATE", "external_knowledge": None},
    {"instance_id": "sf035", "instruction": "How many unique users started sessions each day within each app group between June 1, 2023, and June 7, 2023? Also show the app group ID and the start day of the session.", "db_id": "BRAZE_USER_EVENT_DEMO_DATASET", "external_knowledge": None},
    {"instance_id": "sf029", "instruction": "Generate a daily detailed sales report for each product under the 'Manufacturing' distributor view, covering the 30 days leading up to February 6, 2022. The report should include the total and average values for sales units, revenue, average selling price (ASP), glance views, conversion rate, shipped units, shipped revenue, net profit margin (PPM), and inventory details.", "db_id": "AMAZON_VENDOR_ANALYTICS__SAMPLE_DATASET", "external_knowledge": None},
    {"instance_id": "sf009", "instruction": "A real estate company is looking for a comparison of the building types in Amsterdam and Rotterdam. They need to know the total surface area and the number of buildings for each type of building in both cities. Can you provide the building class and subclass, along with the total surface area and the number of buildings for both Amsterdam and Rotterdam?", "db_id": "NETHERLANDS_OPEN_MAP_DATA", "external_knowledge": None},
    {"instance_id": "sf013", "instruction": "Determine the total length of roads for each class and subclass in Amsterdam and Rotterdam, based on specific QUADKEY segments '12020210' and '12020211'? Show the class, subclass, and total road lengths for both cities", "db_id": "NETHERLANDS_OPEN_MAP_DATA", "external_knowledge": None},
    {"instance_id": "sf041", "instruction": "Produce a report for ERCOT on October 1, 2022, that combines hourly data on day-ahead and real-time prices from node ID 10000697078, load forecasts (datatypeid 19060) and actual loads, plus wind (forecast datatypeid 9285, actual datatypeid 16) and solar (forecast datatypeid 662, actual datatypeid 650) generation forecasts and actuals from object ID 10000712973. This report should include time zone alignments, peak classifications, and net load calculations, providing insights into daily operational dynamics and efficiency.", "db_id": "YES_ENERGY__SAMPLE_DATA", "external_knowledge": "ERCOT_Daily_Market_Dynamics_Report.md"},
    
]

# Base directory
base_path = "/Users/paolocadei/Documents/Masters/Thesis/Spider2/spider2-snow/evaluation_suite/gold/exec_result"

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def process_line(line):
    try:
            
        try:

            
            checker_assistant = create_assistant(
                name="Snowflake Checker",
                instructions=checker_instructions,
                model="o3-mini-2025-01-31",
                client = client
            )

            generator_assistant = create_assistant(
                name="Snowflake Generator",
                instructions=sql_generator_instructions,
                model= "o3-mini-2025-01-31", #"gpt-4.1-2025-04-14"#
                client = client
            )

            question_id = line["instance_id"]
            database = line["db_id"]
            user_prompt = line["instruction"]
            document =line["external_knowledge"]
            doc_path = f"/Users/paolocadei/Documents/Masters/Thesis/Spider2/spider2-snow/resource/documents/{document}"

            print(f"\n\n\n\n\n {question_id} \n\n\n\n\n")

            if document is not None:
                with open(doc_path, 'r') as f:
                    external_knowledge = f.read()

            #print('Starting the process...')
            # Submit all 3 calls in parallel
            with ThreadPoolExecutor() as executor:
                future_low = executor.submit(open_ai_call, client, model, system_prompt, user_prompt)
                #future_mid = executor.submit(open_ai_call, client, model, system_prompt, user_prompt, 500, 0.5, 0.2, 0.4)
                #future_high = executor.submit(open_ai_call, client, model, system_prompt, user_prompt, 500, 0.9, 0.2, 0.4)

                r_low_temp = future_low.result()
                #r_mid_temp = future_mid.result()
                #r_high_temp = future_high.result()

            '''# Print the results
            print(r_low_temp)
            print(r_mid_temp)
            print(r_high_temp)'''

            #print("Reformulations received. Parsing and extracting entities...")

            info = {
                "entities": [],
                "synonyms": [],
                "metrics": [],
                "time_expressions": [],
                "conditions": [],
                "other_keywords": []
            }

            # First, parse all reformulations
            parsed_reformulations = [clean_and_parse_response(r) for r in [r_low_temp]] #, r_mid_temp, r_high_temp]]

            # Create all prompts beforehand
            prompts = [
                'Original question:\n' + user_prompt + '\n\nReformulations:\n' + '\n'.join(r['intent_paraphrases'])
                for r in parsed_reformulations
            ]

            # Parallel execution of open_ai_call
            with ThreadPoolExecutor() as executor:
                futures = [
                    executor.submit(open_ai_call, client, "gpt-4.1-2025-04-14", system_prompt2, prompt)
                    for prompt in prompts
                ]
                responses = [f.result() for f in futures]

            # Parse and merge responses
            for response in responses:
                clean_response = clean_and_parse_response(response)
                if clean_response:
                    for key in info:
                        info[key].extend(clean_response.get(key, []))
                else:
                    print("Missing or invalid response structure")

            # Deduplicate
            for key in info:
                info[key] = list(set(info[key]))

            reformulation_batches = [r_low_temp] #, r_mid_temp, r_high_temp]

            # Step 1: Prepare the embeddings list and Prepare keywords from extracted info

            embeddings, keywords = prepare_embeddings_and_keywords(
                user_prompt=user_prompt,
                reformulation_batches=reformulation_batches,
                info=info,
                embedding_model=EMBEDDING_MODEL,
                openai_client=client,
                get_embedding_func=get_embedding,
                clean_and_parse_func=clean_and_parse_response,
                verbose=False
            )

            # Step 2: Define reusable filters

            qdrant_filter_with_should, qdrant_filter_must_only = get_qdrant_filters(database, keywords)

            points_sorted = accumulate_qdrant_points(
                qdrant_client=qdrant_client,
                collection_name='thesis',
                embeddings=embeddings,
                qdrant_filter_with_should=qdrant_filter_with_should,
                qdrant_filter_must_only=qdrant_filter_must_only,
                keywords=keywords,
                boost_points_func=boost_points,
                boost_factor=1.1,
                verbose=False,
                limit = 150
            )

            # Step 3: Run reranking using Cohere

            rerank_results = rerank_points_with_cohere(
                points_sorted=points_sorted,
                user_prompt=user_prompt,
                cohere_key=cohere_key,
                top_n=75,
                verbose=False
            )

            from pprint import pprint

            # Step 4: Extract top points information

            top_points = extract_top_points_info(
                rerank_results=rerank_results,
                original_points=points_sorted,
                verbose=False
            )

            for item in top_points:
                if 'column' in item and 'column_name' not in item:
                    item['column_name'] = item['column']

            # Step 1: Extract one table per group
            table_mappings = extract_one_table_per_group(top_points)

            conn = snowflake.connector.connect(
                user='pablo_ca',
                password=api_keys["snowflake"],
                account='RSRSBDK-YDB67606'         # Optional if your queries specify schema
            )
            cs = conn.cursor()

            # Step 3: Query Snowflake for column names
            column_lookup = fetch_column_names_parallel(table_mappings, conn_factory, max_workers=10)

            # Step 3.5: Fetch sample values
            sample_lookup = fetch_sample_values_from_column_lookup_parallel(column_lookup, conn_factory, max_workers=30)

            # Step 2: Inject sample values after normalization
            top_points = inject_sample_values_into_top_points(top_points, sample_lookup)

            for item in top_points:
                print(f"{item['column_name']} → sample: {item.get('sample_values')}")

            # Getting the exact column names and format for the output
            output_columns = get_column_names(base_path, question_id)

            # Step 3: Build prompt
            final_prompt = build_prompt_from_points(user_prompt, top_points, column_lookup=column_lookup, expected_output_columns=output_columns)

            if document != None:
                final_prompt += f"\n\nThis is the external knowledge that can used to answer the question: {external_knowledge}\n\n IMPORTANT NOTE: The external knowledge is not a database, only use columns that have been provided above even though. It should be used only to enrich the column information above."

            column_lookup = fetch_column_names_parallel(table_mappings, conn_factory, max_workers=10)

            lines = []
            for (db, schema, table), columns in column_lookup.items():
                col_list = ", ".join(columns)
                lines.append(f"- `{db}.{schema}.{table}`: {col_list}")

            formatted_lookup = "\n".join(lines)
            final_prompt += f"The following shows the perfect and correct tables and columns that can be found:\n{formatted_lookup}"

            # Assuming final_prompt is a string and question_id is a string or number
            file_path = f"/Users/paolocadei/Documents/Masters/Thesis/Spider2/prompts/{question_id}.txt"

            with open(file_path, 'w') as f:
                f.write(final_prompt)

        except Exception as e:
            print(e)
            return

        # Usage in main script
        generator_thread_id, checker_thread_id = create_new_threads(client)

        # Example usage in assistant logic
        message = client.beta.threads.messages.create(
            thread_id=generator_thread_id,
            role="user",
            content=final_prompt
        )

        correct_path = "/Users/paolocadei/Documents/Masters/Thesis/Spider2/7_answers"
        wrong_path = "/Users/paolocadei/Documents/Masters/Thesis/Spider2/7_answers_wrong"
        middle_path = "/Users/paolocadei/Documents/Masters/Thesis/Spider2/7_answers_middle"

        count = 0
        was_valid = False
        last_sql_query = None
        checker_sql = None  # track checker-generated query if used

        while count <= 3:

            '''# Always create a fresh checker thread per round
            checker_thread = client.beta.threads.create()
            checker_thread_id = checker_thread.id'''

            print(f"\n=== Iteration {count + 1} ===")
            sql_query = None
            results = None
            result = None
            checker_results = None
            checker_sql_query = None
            checker_answer = None


            # ATTEMPTS TO GET A PROPERLY FORMATTED SQL QUERY IN 3 ATTEMPTS FROM THE GENERATOR ASSISTANT
            for attempt in range(3):
                print(f"Attempt {attempt + 1}/3")
                raw_text = get_assistant_response(
                    content=final_prompt,
                    thread_id=generator_thread_id,
                    assistant_id=generator_assistant.id,
                    client=client,
                    role_name="generator"
                )

                if not raw_text:
                    print("No response from generator assistant. Retrying...")
                    continue

                sql_query = try_extract_sql(raw_text)
                if sql_query:
                    print("Received SQL query from generator assistant:")
                    print(sql_query)
                    break

                print("Failed to extract SQL query. Re-asking with corrected prompt.")
                final_prompt = create_prompt("Missing SQL Delimiters", expected_output_columns=output_columns)
                

            else:
                print("")
                print("Failed to get a properly formatted SQL query after 3 attempts.")

            if not sql_query:
                print("no valid sql extracted")
                return
            

        #### TRYING TO EXECUTE THE SQL QUERY ON SNOWFLAKE

            status, results, result = execute_sql_with_timeout(cs, sql_query)

            if status in ("success", "timeout"):
                last_sql_query = sql_query
                final_path = os.path.join(
                    middle_path, f"{question_id}.sql"
                )
                with open(final_path, "w") as f:
                    f.write(last_sql_query)
                print(f"Final query saved to {final_path} ('middle')")



            if status != "error big":

                try:

        #### AFTER CHECKING IF THE QUERY WAS STILL RUNNING, WE GET A QUERY FOR THE CHECKER ASSISTANT


        #### CASE IF RESULTS WERE SUCCESSFULLY RETURNED BUT IT MIGHT HAPPEN THAT THEY ARE TOO LONG FOR THE CHECKER ASSISTANT CONTEXT TO PROCESS THEM

                #### CHECKING THE OUTPUT OF THE CHECKER ASSISTANT

                    terminate = False
                    checker_answer = None
                    if status == "success":

                        for attempt in range(3):

                            print(f"Checker attempt {attempt + 1}/3")

                            if attempt > 0 and len(results) > 1:
                                results = results[:len(results) // 2]

                            if len(result) == 0:
                                percentage_of_results = 0.0
                            else:
                                percentage_of_results = len(results) / len(result) * 100


                            content = create_prompt(status, sql_query=sql_query, results=results, question=user_prompt, percentage_of_results=percentage_of_results, expected_output_columns=output_columns)

                            '''# Always create a fresh checker thread per round
                            checker_thread = client.beta.threads.create()
                            checker_thread_id = checker_thread.id'''

                            checker_answer = get_assistant_response(content=content, thread_id=checker_thread_id, assistant_id=checker_assistant.id, client=client, role_name="checker")

                            if checker_answer:
                                print("Checker assistant responded successfully.")
                                print(checker_answer)
                                break
                            else:
                                print("")
                                print("No usable response from checker assistant.")


                        else:
                            # This runs if the loop completes without a break
                            print("Checker assistant failed to provide useful output after 3 attempts.")
                            terminate = True
                            count += 1

                        ### NEED TO ADD SAVING MECHANISM FOR THE SQL QUERY AS IT WAS CORRECT

                    if status == "error":
                        # Check for invalid column qualifier
                        error_message = results.lower()
                        if "invalid identifier" in error_message or "invalid column" in error_message:
                            content = create_prompt(
                                "error_with_columns",
                                sql_query=sql_query,
                                results=results,
                                question=user_prompt,
                                column_lookup=column_lookup,
                                expected_output_columns=output_columns
                            )

                            checker_answer = get_assistant_response(
                                content=content,
                                thread_id=checker_thread_id,
                                assistant_id=checker_assistant.id,
                                client=client,
                                role_name="checker"
                            )
                        else:
                            content = create_prompt(
                                    status,
                                    sql_query=sql_query,
                                    results=results,
                                    question=user_prompt,
                                    expected_output_columns=output_columns
                                )

                            checker_answer = get_assistant_response(
                                content=content,
                                thread_id=checker_thread_id,
                                assistant_id=checker_assistant.id,
                                client=client,
                                role_name="checker"
                            )

                    
                    if status == "empty":

                        content = create_prompt(status, sql_query=sql_query, question=user_prompt, column_lookup=column_lookup, expected_output_columns=output_columns)

                        checker_answer = get_assistant_response(content=content, thread_id=checker_thread_id, assistant_id=checker_assistant.id, client=client, role_name="checker")

                    if status == "timeout":
                        
                        content = create_prompt(status, sql_query=sql_query, question=user_prompt, expected_output_columns=output_columns)

                        checker_answer = get_assistant_response(content=content, thread_id=checker_thread_id, assistant_id=checker_assistant.id, client=client, role_name="checker")


            #### SAVING THE DIFFERENET CASES OF THE CHECKER ASSISTANT AS FINAL PROMPT FOR THE GENERATOR ASSISTANT

                    print(f"Checker assistant outputted the following response: \n{checker_answer}.")
                    final_prompt = checker_answer

            #### CHECKING IF CHECKER ASSISTANT ANSWERED WITH A VALID SQL QUERY OR IF IT SAID "DONE"

                    checker_sql_query = try_extract_sql(checker_answer)
                    if checker_sql_query:
                        print("Received SQL query from checker assistant:")
                        checker_status, checker_results, checker_result = execute_sql_with_timeout(cs, checker_sql_query)

                        if checker_status in ("success", "timeout"):
                            print("Checker SQL executed successfully. Saved as correct query.")
                            last_sql_query = checker_sql_query
                            final_path = os.path.join(
                                middle_path, f"{question_id}.sql"
                            )
                            with open(final_path, "w") as f:
                                f.write(last_sql_query)
                            print(f"Final query saved to {final_path} ('middle')")

                            #### CHECKING IF THE CHECKER ASSISTANT IS SATISFIED WITH THE SQL QUERY AND SAYS "DONE"

                            percentage_of_results = (len(checker_results) / len(checker_result) * 100) if checker_result else 0.0

                            content = create_prompt(
                                "checker_correct",
                                sql_query=checker_sql_query,
                                results=checker_results,
                                percentage_of_results=percentage_of_results,
                                expected_output_columns=output_columns
                            )
                            
                            checker_answer = get_assistant_response(content=content, thread_id=checker_thread_id, assistant_id=checker_assistant.id, client=client, role_name="checker")

                            if checker_answer:

                                final_prompt = checker_answer


                        if checker_status in ("error", "empty"):
                            content = create_prompt("checker_second_attempt", sql_query=sql_query, question=user_prompt, results = results, column_lookup=column_lookup, expected_output_columns=output_columns)

                            checker_answer = get_assistant_response(content=content, thread_id=checker_thread_id, assistant_id=checker_assistant.id, client=client, role_name="checker")

                            if checker_answer:
                                final_prompt = checker_answer
                            else:
                                final_prompt = create_prompt("checker_no_second_attempt", results = results, checker_results = checker_results, checker_sql_query=checker_sql_query, expected_output_columns=output_columns)

                    else:
                        print("")
                        print("Checker assistant did not return a valid SQL query.")

            #### CHECKING IF THE CHECKER ASSISTANT THINKS THAT THE SQL QUERY IS CORRECT AND IS FINAL

                    if "query validated. marking as final." in checker_answer.lower():
                        final_prompt = checker_answer
                        print("Checker marked query as correct. Saving and exiting.")
                        filepath = os.path.join(correct_path, f"{question_id}.sql")
                        with open(filepath, "w") as f:
                            f.write(last_sql_query.strip())
                        was_valid = True
                        break
                    else:
                        print("Checker did not say 'Done'. Passing feedback to generator.")
                        count += 1
                        continue

                except Exception as e:
                    print(f"⚠️ An exception occurred in checker-validation logic: {e}")
                    print("Attempting to recover using last_sql_query from middle path...")

                    if last_sql_query is not None:
                        final_path = os.path.join(
                            correct_path, f"{question_id}.sql"
                        )
                        with open(final_path, "w") as f:
                            f.write(last_sql_query)
                        print(f"Query recovered from middle and saved to {final_path} (wrong)")
                        count += 1
                    else:
                        print("No fallback last_sql_query available. Skipping saving.")
                        count += 1

            else:

                print("Some king of error occurred during the process")
                print(results)
                count += 1

        if last_sql_query is not None:
            final_path = os.path.join(
                correct_path if was_valid else wrong_path, f"{question_id}.sql"
            )
            with open(final_path, "w") as f:
                f.write(last_sql_query)
            print(f"Final query saved to {final_path} ({'correct' if was_valid else 'wrong'})")

        print("Closing Snowflake connection.")
        cs.close()
        conn.close()
        print("Done.")
    except Exception as e:
        print(f"⚠️ Error processing line {line.get('instance_id', '?')}: {e}")

for line in lines:
    process_line(line)

Creating new assistant 'Snowflake Checker'...
Creating new assistant 'Snowflake Generator'...





 sf_local272 







Fetching samples: 100%|██████████| 82/82 [00:36<00:00,  2.24it/s]


⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERLINES') → column 'product_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'qty'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'product_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'location_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PURCHASES') → column 'product_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERLINES') → column 'qty'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERS') → column 'id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERLINES') → column 'id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PURCHASES') → column 'qty'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'purchase_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PRODUCTS') → column 'id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERS') → column 'ordered'
⚠️ No match for ('

Fetching samples: 100%|██████████| 77/77 [00:33<00:00,  2.29it/s]


⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'qty'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERS') → column 'ordered'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PURCHASES') → column 'qty'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PURCHASES') → column 'cost'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERLINES') → column 'qty'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'ORDERLINES') → column 'amount'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PURCHASES') → column 'purchased'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'location_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'product_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PRODUCTS') → column 'id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'PURCHASES') → column 'product_id'
⚠️ No match for ('ORACLE_SQL', 'ORACLE_SQL', 'INVENTORY') → column 'purchase_id'
⚠️ No match

...

⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_LOCALIZED_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_LOCALIZED_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_SYMBOL'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_3_LOCALIZED_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_3_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_WIKIDATA'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_NAME'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_WIKIDATA'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → colum

Fetching samples: 100%|██████████| 91/91 [00:42<00:00,  2.16it/s]


⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_3_NETWORK'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_3_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_REF'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_ROUTE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_NETWORK'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_3_LOCALIZED_TYPE'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_2_NETWORK'
⚠️ No match for ('NETHERLANDS_OPEN_MAP_DATA', 'NETHERLANDS', 'V_ROAD') → column 'REL_TIER_1_REF'
⚠️

Fetching samples: 100%|██████████| 192/192 [01:26<00:00,  2.21it/s]


⚠️ No match for ('YES_ENERGY__SAMPLE_DATA', 'YES_ENERGY_SAMPLE', 'RT_PRICES_RECENT_SAMPLE') → column 'CONG'
⚠️ No match for ('YES_ENERGY__SAMPLE_DATA', 'YES_ENERGY_SAMPLE', 'DART_PRICES_SAMPLE') → column 'DACONG'
⚠️ No match for ('YES_ENERGY__SAMPLE_DATA', 'YES_ENERGY_SAMPLE', 'RT_PRICES_RECENT_SAMPLE') → column 'LOSS'
⚠️ No match for ('YES_ENERGY__SAMPLE_DATA', 'YES_ENERGY_SAMPLE', 'DART_PRICES_SAMPLE') → column 'PROXYCODE'
INTERVAL → sample: [900, 300, 3600, 10]
INTEGRATION_RULE → sample: ['UP', 'DOWN']
SERIESNAME → sample: ['DA_ASMVOL_REG_D', 'ASM_DA_ECRS', 'RTLMP', 'ASM_DA_RRS', 'SYSTEM_LAMBDA_DA', 'RT_ON_CAP', 'RTLOAD', 'DALMP', 'WIND_STWPF', 'GENERATION_SOLAR_RT']
DATASOURCE → sample: ['ERCOT']
TIMEZONE → sample: ['CST', 'CDT']
SOURCE → sample: ['ERCOT']
DATATYPEID → sample: [21, 22, 23, 24, 138, 9040, 19, 9139, 17, 9076]
DATATYPEID → sample: [19, 661, 1034, 1036, 9285, 662, 10555, 9139, 138, 19060]
TIMEZONE → sample: ['CST', 'CDT']
TIMEZONE → sample: ['CDT', 'CST']
DATATYPEID → 